##CASE STUDY 1
Title: Order Processing and Analytics Pipeline using PySpark

A retail company operates in multiple cities and receives daily order data from its transactional
systems.

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

In [2]:
spark = SparkSession.builder.appName("Capstone One").getOrCreate()

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
csv_path ="/content/drive/MyDrive/Colab Notebooks/orders.csv"

##PHASE 1 – Data Ingestion

1. Load the CSV le without schema inference.
2. Print the schema.
3. Count total records.
4. Show sample rows.

In [5]:
df_csv_raw = (spark.read.option("header", "True")
              .option("inferSchema", "False").csv(csv_path))
df_csv_raw.printSchema()
df_csv_raw.count()
df_csv_raw.show(10, truncate=False)

root
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- city: string (nullable = true)
 |-- category: string (nullable = true)
 |-- product: string (nullable = true)
 |-- amount: string (nullable = true)
 |-- order_date: string (nullable = true)
 |-- status: string (nullable = true)

+-----------+-----------+-----------+-----------+-----------+-------+----------+---------+
|order_id   |customer_id|city       |category   |product    |amount |order_date|status   |
+-----------+-----------+-----------+-----------+-----------+-------+----------+---------+
|ORD00000000|C000000    | hyderabad | grocery   |Oil        |invalid|01/01/2024|Cancelled|
|ORD00000001|C000001    |Pune       |Grocery    |Sugar      |35430  |2024-01-02|Completed|
|ORD00000002|C000002    |Pune       |Electronics|Mobile     |65358  |2024-01-03|Completed|
|ORD00000003|C000003    |Bangalore  |Electronics|Laptop     |5558   |2024-01-04|Completed|
|ORD00000004|C000004    |Pune       |Home

5. Explain why all columns must be treated as StringType initially.

In [6]:
#Reading all columns as StringType initially :
#1 Prevents job failure due to malformed data
#2 Supports multiple date formats
#3 Allows custom validation and error handling

##PHASE 2 – Data Cleaning

1. Remove leading and trailing spaces from:
- city
- category
- product

2. Standardize text:
- Convert city, category, and product to proper case.

In [7]:
df_clean = df_csv_raw \
    .withColumn("city_clean", initcap(trim(lower(col("city"))))) \
    .withColumn("category_clean", initcap(trim(lower(col("category"))))) \
    .withColumn("product_clean", initcap(trim(lower(col("product")))))
df_clean.show(10)

+-----------+-----------+-----------+-----------+-----------+-------+----------+---------+----------+--------------+-------------+
|   order_id|customer_id|       city|   category|    product| amount|order_date|   status|city_clean|category_clean|product_clean|
+-----------+-----------+-----------+-----------+-----------+-------+----------+---------+----------+--------------+-------------+
|ORD00000000|    C000000| hyderabad |   grocery |       Oil |invalid|01/01/2024|Cancelled| Hyderabad|       Grocery|          Oil|
|ORD00000001|    C000001|       Pune|    Grocery|      Sugar|  35430|2024-01-02|Completed|      Pune|       Grocery|        Sugar|
|ORD00000002|    C000002|       Pune|Electronics|     Mobile|  65358|2024-01-03|Completed|      Pune|   Electronics|       Mobile|
|ORD00000003|    C000003|  Bangalore|Electronics|     Laptop|   5558|2024-01-04|Completed| Bangalore|   Electronics|       Laptop|
|ORD00000004|    C000004|       Pune|       Home|AirPurifier|  33659|2024-01-05|Com

3. Clean the amount column:
- Remove commas.
- Replace empty strings and invalid values with null.
- Convert amount into IntegerType.
- Rows with invalid amounts must not crash the pipeline.

In [8]:
df_clean = df_clean.withColumn("amount_clean",regexp_replace(col("amount"), ",", ""))

df_clean = df_clean.withColumn("amount_int",when(col("amount_clean").rlike("^[0-9]+$"),col("amount_clean").cast(IntegerType())).otherwise(None))
df_clean.show(10)

+-----------+-----------+-----------+-----------+-----------+-------+----------+---------+----------+--------------+-------------+------------+----------+
|   order_id|customer_id|       city|   category|    product| amount|order_date|   status|city_clean|category_clean|product_clean|amount_clean|amount_int|
+-----------+-----------+-----------+-----------+-----------+-------+----------+---------+----------+--------------+-------------+------------+----------+
|ORD00000000|    C000000| hyderabad |   grocery |       Oil |invalid|01/01/2024|Cancelled| Hyderabad|       Grocery|          Oil|     invalid|      NULL|
|ORD00000001|    C000001|       Pune|    Grocery|      Sugar|  35430|2024-01-02|Completed|      Pune|       Grocery|        Sugar|       35430|     35430|
|ORD00000002|    C000002|       Pune|Electronics|     Mobile|  65358|2024-01-03|Completed|      Pune|   Electronics|       Mobile|       65358|     65358|
|ORD00000003|    C000003|  Bangalore|Electronics|     Laptop|   5558|2

4. Clean the order_date column:
- Support the following formats:
- yyyy-MM-dd
- dd/MM/yyyy
- yyyy/MM/dd

- Create a new column:

order_date_clean
with DateType.

In [9]:
df_clean=df_clean.withColumn("order_date_clean",
    coalesce(
        try_to_timestamp(col("order_date"), lit("yyyy-MM-dd")),
        try_to_timestamp(col("order_date"), lit("dd-MM-yyyy")),
        try_to_timestamp(col("order_date"), lit("MM-dd-yyyy")),
        try_to_timestamp(col("order_date"), lit("dd/MM/yyyy")),
        try_to_timestamp(col("order_date"), lit("MM/dd/yyyy")),
        try_to_timestamp(col("order_date"), lit("yyyy/MM/dd"))
    ).cast(DateType())
)
df_clean.show()

+-----------+-----------+-----------+-----------+-----------+-------+----------+---------+----------+--------------+-------------+------------+----------+----------------+
|   order_id|customer_id|       city|   category|    product| amount|order_date|   status|city_clean|category_clean|product_clean|amount_clean|amount_int|order_date_clean|
+-----------+-----------+-----------+-----------+-----------+-------+----------+---------+----------+--------------+-------------+------------+----------+----------------+
|ORD00000000|    C000000| hyderabad |   grocery |       Oil |invalid|01/01/2024|Cancelled| Hyderabad|       Grocery|          Oil|     invalid|      NULL|      2024-01-01|
|ORD00000001|    C000001|       Pune|    Grocery|      Sugar|  35430|2024-01-02|Completed|      Pune|       Grocery|        Sugar|       35430|     35430|      2024-01-02|
|ORD00000002|    C000002|       Pune|Electronics|     Mobile|  65358|2024-01-03|Completed|      Pune|   Electronics|       Mobile|       653

##PHASE 3 – Data Validation

1. Count how many records had invalid amounts

In [10]:
invalid_amount_count = df_clean.filter(col("amount_int").isNull()).count()
print("Invalid Amount Count:", invalid_amount_count)

Invalid Amount Count: 25164


2. Count how many records had invalid dates.

In [11]:
invalid_date_count = df_clean.filter(col("order_date_clean").isNull()).count()
print("Invalid Date Count:", invalid_date_count)

Invalid Date Count: 2595


3. Identify duplicate order_id values.

In [12]:
duplicate_orders = df_clean.groupBy("order_id").count().filter(col("count") > 1)

4. Remove duplicates using order_id.

In [13]:
df_dedup = df_clean.dropDuplicates(["order_id"])
df_dedup.show()

+-----------+-----------+-----------+-----------+-----------+-------+----------+---------+----------+--------------+-------------+------------+----------+----------------+
|   order_id|customer_id|       city|   category|    product| amount|order_date|   status|city_clean|category_clean|product_clean|amount_clean|amount_int|order_date_clean|
+-----------+-----------+-----------+-----------+-----------+-------+----------+---------+----------+--------------+-------------+------------+----------+----------------+
|ORD00000001|    C000001|       Pune|    Grocery|      Sugar|  35430|2024-01-02|Completed|      Pune|       Grocery|        Sugar|       35430|     35430|      2024-01-02|
|ORD00000007|    C000007|       Pune|    Grocery|       Rice|  45362|2024-01-08|Completed|      Pune|       Grocery|         Rice|       45362|     45362|      2024-01-08|
|ORD00000008|    C000008|  Bangalore|    Fashion|      Jeans|  10563|2024-01-09|Completed| Bangalore|       Fashion|        Jeans|       105

5. Filter only records with:
status = "Completed"

In [14]:
df_final = df_dedup.filter(lower(col("status")) == "completed")
df_final.show(25)

+-----------+-----------+-----------+-----------+-----------+-------+----------+---------+----------+--------------+-------------+------------+----------+----------------+
|   order_id|customer_id|       city|   category|    product| amount|order_date|   status|city_clean|category_clean|product_clean|amount_clean|amount_int|order_date_clean|
+-----------+-----------+-----------+-----------+-----------+-------+----------+---------+----------+--------------+-------------+------------+----------+----------------+
|ORD00000001|    C000001|       Pune|    Grocery|      Sugar|  35430|2024-01-02|Completed|      Pune|       Grocery|        Sugar|       35430|     35430|      2024-01-02|
|ORD00000007|    C000007|       Pune|    Grocery|       Rice|  45362|2024-01-08|Completed|      Pune|       Grocery|         Rice|       45362|     45362|      2024-01-08|
|ORD00000008|    C000008|  Bangalore|    Fashion|      Jeans|  10563|2024-01-09|Completed| Bangalore|       Fashion|        Jeans|       105

6. Record row counts at every stage

In [15]:
print("Raw:", df_csv_raw.count())
print("After Deleted duplicate:", df_dedup.count())
print("Final:", df_final.count())

Raw: 300000
After Deleted duplicate: 300000
Final: 285000


##PHASE 4 – Performance Engineering

1. Check the number of partitions.

In [16]:
df_final.rdd.getNumPartitions()

2

2. Run a groupBy on city and calculate total revenue.

In [17]:
city_revenue = df_final.groupBy("city_clean").agg(sum("amount_int").alias("total_revenue"))
city_revenue.show()

+----------+-------------+
|city_clean|total_revenue|
+----------+-------------+
| Bangalore|   1628527093|
|   Chennai|   1629865247|
|    Mumbai|   1625518096|
|   Kolkata|   1624300497|
|      Pune|   1646196535|
|     Delhi|   1639639916|
| Hyderabad|   1642443340|
+----------+-------------+



5. Repartition the dataset by city.

In [18]:
df_repart = df_final.repartition("city_clean")
city_revenue_after = df_repart.groupBy("city_clean").sum("amount_int")
city_revenue_after.show()

+----------+---------------+
|city_clean|sum(amount_int)|
+----------+---------------+
| Bangalore|     1628527093|
|   Chennai|     1629865247|
|    Mumbai|     1625518096|
|   Kolkata|     1624300497|
|      Pune|     1646196535|
|     Delhi|     1639639916|
| Hyderabad|     1642443340|
+----------+---------------+



3. Use:
explain(True) to analyze execution.
4. Identify where shuffle happens.
6. Compare execution plans before and after repartition.

In [19]:
city_revenue.explain(True)
city_revenue_after.explain(True)

== Parsed Logical Plan ==
'Aggregate ['city_clean], ['city_clean, 'sum('amount_int) AS total_revenue#887]
+- Filter (lower(status#24) = completed)
   +- Deduplicate [order_id#17]
      +- Project [order_id#17, customer_id#18, city#19, category#20, product#21, amount#22, order_date#23, status#24, city_clean#71, category_clean#72, product_clean#73, amount_clean#116, amount_int#117, cast(coalesce(try_to_timestamp(order_date#23, Some(yyyy-MM-dd), TimestampType, Some(Etc/UTC), false), try_to_timestamp(order_date#23, Some(dd-MM-yyyy), TimestampType, Some(Etc/UTC), false), try_to_timestamp(order_date#23, Some(MM-dd-yyyy), TimestampType, Some(Etc/UTC), false), try_to_timestamp(order_date#23, Some(dd/MM/yyyy), TimestampType, Some(Etc/UTC), false), try_to_timestamp(order_date#23, Some(MM/dd/yyyy), TimestampType, Some(Etc/UTC), false), try_to_timestamp(order_date#23, Some(yyyy/MM/dd), TimestampType, Some(Etc/UTC), false)) as date) AS order_date_clean#166]
         +- Project [order_id#17, custome

##PHASE 5 – Analytics

1. Total revenue per city.
2. Total revenue per category.
3. Average order value per city.
4. Top 10 products by revenue.
5. Cities sorted by revenue descending.

In [20]:
#1
df_final.groupBy("city_clean").sum("amount_int").show()

#2
df_final.groupBy("category_clean").sum("amount_int").show()

#3
df_final.groupBy("city_clean").avg("amount_int").show()

#4
df_final.groupBy("product_clean").sum("amount_int").orderBy(desc("sum(amount_int)")).limit(10).show()

#5
city_revenue.orderBy(col("total_revenue").desc()).show(5)

+----------+---------------+
|city_clean|sum(amount_int)|
+----------+---------------+
| Bangalore|     1628527093|
|   Chennai|     1629865247|
|    Mumbai|     1625518096|
|   Kolkata|     1624300497|
|      Pune|     1646196535|
|     Delhi|     1639639916|
| Hyderabad|     1642443340|
+----------+---------------+

+--------------+---------------+
|category_clean|sum(amount_int)|
+--------------+---------------+
|          Home|     2868467576|
|       Fashion|     2834182172|
|       Grocery|     2866272106|
|   Electronics|     2867568870|
+--------------+---------------+

+----------+------------------+
|city_clean|   avg(amount_int)|
+----------+------------------+
| Bangalore|44098.867908689645|
|   Chennai| 43628.27900315863|
|    Mumbai| 43723.75651612556|
|   Kolkata|43709.816662630175|
|      Pune|43930.204013556424|
|     Delhi| 43817.20780331374|
| Hyderabad| 43708.74045293664|
+----------+------------------+

+-------------+---------------+
|product_clean|sum(amount_int)

##PHASE 6 – Window Functions

1. Rank cities by revenue.

In [21]:
city_window = Window.orderBy(desc("total_revenue"))
ranked_cities = city_revenue.withColumn("rank", rank().over(city_window))
ranked_cities.show(5)

+----------+-------------+----+
|city_clean|total_revenue|rank|
+----------+-------------+----+
|      Pune|   1646196535|   1|
| Hyderabad|   1642443340|   2|
|     Delhi|   1639639916|   3|
|   Chennai|   1629865247|   4|
| Bangalore|   1628527093|   5|
+----------+-------------+----+
only showing top 5 rows


2. Rank products inside each category by revenue

In [22]:
category_window = Window.partitionBy("category_clean").orderBy(desc("total_revenue"))

 3. Find the top product for every category.

In [24]:
df_product_revenue = df_final.groupBy("category_clean", "product_clean").agg(sum("amount_int").alias("total_revenue_per_product"))
category_product_window = Window.partitionBy("category_clean").orderBy(desc("total_revenue_per_product"))
df_ranked_products = df_product_revenue.withColumn("rank", rank().over(category_product_window))
df_ranked_products.show()

+--------------+-------------+-------------------------+----+
|category_clean|product_clean|total_revenue_per_product|rank|
+--------------+-------------+-------------------------+----+
|   Electronics|       Laptop|                962496295|   1|
|   Electronics|       Tablet|                960719999|   2|
|   Electronics|       Mobile|                944352576|   3|
|       Fashion|        Jeans|                951286127|   1|
|       Fashion|        Shoes|                946799102|   2|
|       Fashion|       Tshirt|                936096943|   3|
|       Grocery|          Oil|                963572869|   1|
|       Grocery|         Rice|                954494237|   2|
|       Grocery|        Sugar|                948205000|   3|
|          Home|       Vacuum|                959149427|   1|
|          Home|        Mixer|                957140026|   2|
|          Home|  Airpurifier|                952178123|   3|
+--------------+-------------+-------------------------+----+



4. Identify the top 3 performing cities.

In [25]:
ranked_cities.filter(col("rank") <= 3).show()

+----------+-------------+----+
|city_clean|total_revenue|rank|
+----------+-------------+----+
|      Pune|   1646196535|   1|
| Hyderabad|   1642443340|   2|
|     Delhi|   1639639916|   3|
+----------+-------------+----+



##PHASE 7 – Broadcast Join

A small lookup table is provided:

city,region

Delhi,North

Mumbai,West

Bangalore,South

Hyderabad,South

Pune,West

Chennai,South

Kolkata,East

Tasks:
1. Join the orders data with this city-region dataset.
2. Apply broadcast join explicitly.

In [26]:
city_region = spark.createDataFrame([
    ("Delhi", "North"),
    ("Mumbai", "West"),
    ("Bangalore", "South"),
    ("Hyderabad", "South"),
    ("Pune", "West"),
    ("Chennai", "South"),
    ("Kolkata", "East")], ["city", "region"])

joined_df = df_final.join(broadcast(city_region),df_final.city_clean == city_region.city,"left")
joined_df.show()

+-----------+-----------+-----------+-----------+-----------+-------+----------+---------+----------+--------------+-------------+------------+----------+----------------+---------+------+
|   order_id|customer_id|       city|   category|    product| amount|order_date|   status|city_clean|category_clean|product_clean|amount_clean|amount_int|order_date_clean|     city|region|
+-----------+-----------+-----------+-----------+-----------+-------+----------+---------+----------+--------------+-------------+------------+----------+----------------+---------+------+
|ORD00000001|    C000001|       Pune|    Grocery|      Sugar|  35430|2024-01-02|Completed|      Pune|       Grocery|        Sugar|       35430|     35430|      2024-01-02|     Pune|  West|
|ORD00000007|    C000007|       Pune|    Grocery|       Rice|  45362|2024-01-08|Completed|      Pune|       Grocery|         Rice|       45362|     45362|      2024-01-08|     Pune|  West|
|ORD00000008|    C000008|  Bangalore|    Fashion|      

3. Verify using the physical plan that:
BroadcastHashJoin

In [27]:
joined_df.explain(True)

== Parsed Logical Plan ==
Join LeftOuter, (city_clean#71 = city#1935)
:- Filter (lower(status#24) = completed)
:  +- Deduplicate [order_id#17]
:     +- Project [order_id#17, customer_id#18, city#19, category#20, product#21, amount#22, order_date#23, status#24, city_clean#71, category_clean#72, product_clean#73, amount_clean#116, amount_int#117, cast(coalesce(try_to_timestamp(order_date#23, Some(yyyy-MM-dd), TimestampType, Some(Etc/UTC), false), try_to_timestamp(order_date#23, Some(dd-MM-yyyy), TimestampType, Some(Etc/UTC), false), try_to_timestamp(order_date#23, Some(MM-dd-yyyy), TimestampType, Some(Etc/UTC), false), try_to_timestamp(order_date#23, Some(dd/MM/yyyy), TimestampType, Some(Etc/UTC), false), try_to_timestamp(order_date#23, Some(MM/dd/yyyy), TimestampType, Some(Etc/UTC), false), try_to_timestamp(order_date#23, Some(yyyy/MM/dd), TimestampType, Some(Etc/UTC), false)) as date) AS order_date_clean#166]
:        +- Project [order_id#17, customer_id#18, city#19, category#20, produ

4. Explain why broadcast join is e cient in this case

In [28]:
#Broadcast join is efficient because
# 1.Small lookup table
# 2.Avoids shuffle on large dataset

##PHASE 8 – UDF
- Create a classi cation based on amount:

amount >= 80000  → High

amount >= 40000  → Medium

else             → Low

- Add a new column:
order_value_category
- Analyze distribution.

In [29]:
def classify(amount):
    if amount is None:
        return None
    if amount >= 80000:
        return "High"
    elif amount >= 40000:
        return "Medium"
    else:
        return "Low"

udf_classify = udf(classify, StringType())
df_final = df_final.withColumn("order_value_category",udf_classify(col("amount_int")))
df_final.show()

+-----------+-----------+-----------+-----------+-----------+-------+----------+---------+----------+--------------+-------------+------------+----------+----------------+--------------------+
|   order_id|customer_id|       city|   category|    product| amount|order_date|   status|city_clean|category_clean|product_clean|amount_clean|amount_int|order_date_clean|order_value_category|
+-----------+-----------+-----------+-----------+-----------+-------+----------+---------+----------+--------------+-------------+------------+----------+----------------+--------------------+
|ORD00000001|    C000001|       Pune|    Grocery|      Sugar|  35430|2024-01-02|Completed|      Pune|       Grocery|        Sugar|       35430|     35430|      2024-01-02|                 Low|
|ORD00000007|    C000007|       Pune|    Grocery|       Rice|  45362|2024-01-08|Completed|      Pune|       Grocery|         Rice|       45362|     45362|      2024-01-08|              Medium|
|ORD00000008|    C000008|  Bangalor

In [30]:
df_final.groupBy("order_value_category").count().show()

+--------------------+------+
|order_value_category| count|
+--------------------+------+
|                High| 27936|
|                 Low|121794|
|                NULL| 23905|
|              Medium|111365|
+--------------------+------+



##PHASE 9 – RDD

1. Convert the cleaned DataFrame to RDD

In [31]:
orders_rdd = df_final.select("city_clean", "amount_int").rdd

2. Compute:
- Total revenue using reduce.
- Orders per city using map and reduce.

In [33]:
total_revenue = orders_rdd.map(lambda x: x[1] if x[1] is not None else 0).reduce(lambda x, y: x + y)
print("Total revenue:", total_revenue)

Total revenue: 11436490724


3. Explain why DataFrames are preferred over RDDs for analytics.

In [34]:
#DataFrames are faster and optimized using Catalyst and Tungsten.
#Support SQL queries and rich built-in functions.
#Schema enforcement ensures structured data and better performance.
#Columnar storage improves memory and speed.

##PHASE 10 – Caching

1. Identify datasets reused in multiple queries.

In [35]:
# Datasets heavily reused in multiple queries:
# 1. df_final: Used for all analytics (Phase 5), window functions (Phase 6), broadcast joins (Phase 7), UDF (Phase 8), and RDD conversions (Phase 9).
# 2. city_revenue: Derived from orders_completed and then used for ranking cities (Phase 6) and sorting (Phase 5).

print("Datasets identified for potential caching:")
print("- orders_completed (used extensively for analytics, joins, and RDD operations)")
print("- city_revenue (used for further analysis and window functions)")

Datasets identified for potential caching:
- orders_completed (used extensively for analytics, joins, and RDD operations)
- city_revenue (used for further analysis and window functions)


2. Apply cache().

In [36]:
df_final.cache()
city_revenue.cache()

print("orders_completed DataFrame is cached:", df_final.is_cached)
print("city_revenue DataFrame is cached:", city_revenue.is_cached)

orders_completed DataFrame is cached: True
city_revenue DataFrame is cached: True


3. Execute multiple aggregations.

In [37]:
print("Executing aggregations on cached DataFrames...")

# Aggregation 1: Total revenue per category (on cached orders_completed)
df_final.groupBy("category_clean").agg(sum("amount_int").alias("total_revenue")).show(5)

# Aggregation 2: Average order value per city (on cached orders_completed)
df_final.groupBy("city_clean").agg(avg("amount_int").alias("avg_order_value")).show(5)

# Aggregation 3: Max revenue per city (on cached city_revenue, which itself is cached)
city_revenue.agg(max("total_revenue")).show()

print("Aggregations executed.")

Executing aggregations on cached DataFrames...
+--------------+-------------+
|category_clean|total_revenue|
+--------------+-------------+
|          Home|   2868467576|
|       Fashion|   2834182172|
|       Grocery|   2866272106|
|   Electronics|   2867568870|
+--------------+-------------+

+----------+------------------+
|city_clean|   avg_order_value|
+----------+------------------+
| Bangalore|44098.867908689645|
|   Chennai| 43628.27900315863|
|    Mumbai| 43723.75651612556|
|   Kolkata|43709.816662630175|
|      Pune|43930.204013556424|
+----------+------------------+
only showing top 5 rows
+------------------+
|max(total_revenue)|
+------------------+
|        1646196535|
+------------------+

Aggregations executed.


4. Compare performance.

In [38]:
import time

print("--- Comparing Performance of Cached vs. Uncached DataFrames ---")

# Unpersist DataFrames first to ensure a clean comparison
df_final.unpersist()
city_revenue.unpersist()
print("\nDataFrames unpersisted for baseline comparison.")

# --- Baseline: Execute aggregations without caching ---
print("\nExecuting aggregations WITHOUT caching...")
start_time_uncached = time.time()

df_final.groupBy("category_clean").agg(sum("amount_int").alias("total_revenue")).collect()
df_final.groupBy("city_clean").agg(avg("amount_int").alias("avg_order_value")).collect()
city_revenue.agg(max("total_revenue")).collect()

end_time_uncached = time.time()
duration_uncached = end_time_uncached - start_time_uncached
print(f"Aggregations WITHOUT caching took: {duration_uncached:.4f} seconds")

# --- Re-cache DataFrames ---
df_final.cache()
city_revenue.cache()
# Trigger caching with an action
df_final.count()
city_revenue.count()
print("\nDataFrames re-cached.")

# --- Cached: Execute aggregations WITH caching ---
print("\nExecuting aggregations WITH caching...")
start_time_cached = time.time()

df_final.groupBy("category_clean").agg(sum("amount_int").alias("total_revenue")).collect()
df_final.groupBy("city_clean").agg(avg("amount_int").alias("avg_order_value")).collect()
city_revenue.agg(max("total_revenue")).collect()

end_time_cached = time.time()
duration_cached = end_time_cached - start_time_cached
print(f"Aggregations WITH caching took: {duration_cached:.4f} seconds")

print("\nPerformance Comparison:")
print(f"Uncached duration: {duration_uncached:.4f} seconds")
print(f"Cached duration:   {duration_cached:.4f} seconds")
if duration_uncached > 0:
    speedup = duration_uncached / duration_cached
    print(f"Caching provided a speedup of approximately {speedup:.2f}x")

--- Comparing Performance of Cached vs. Uncached DataFrames ---

DataFrames unpersisted for baseline comparison.

Executing aggregations WITHOUT caching...
Aggregations WITHOUT caching took: 10.6803 seconds

DataFrames re-cached.

Executing aggregations WITH caching...
Aggregations WITH caching took: 9.9177 seconds

Performance Comparison:
Uncached duration: 10.6803 seconds
Cached duration:   9.9177 seconds
Caching provided a speedup of approximately 1.08x


5. Unpersist after use.

Explain why unnecessary caching is dangerous.

In [39]:
df_final.unpersist()

#memory pressure
#executor oom
#cache eviction
#slower jobs

DataFrame[order_id: string, customer_id: string, city: string, category: string, product: string, amount: string, order_date: string, status: string, city_clean: string, category_clean: string, product_clean: string, amount_clean: string, amount_int: int, order_date_clean: date, order_value_category: string]

##PHASE 11 – Storage Formats

1. Write cleaned dataset to:

Parquet
Partitioned by:
city


In [40]:
df_final.write.mode("overwrite").partitionBy("city").parquet("orders_parquet")

2. Write aggregated datasets to:

ORC

In [41]:
city_revenue.write.mode("overwrite").orc("city_revenue_orc")

3. Read both formats back and validate:
- Schema
- Row counts

In [42]:
spark.read.parquet("orders_parquet").printSchema()
spark.read.parquet("orders_parquet").count()
spark.read.orc("city_revenue_orc").printSchema()
spark.read.orc("city_revenue_orc").count()

root
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- category: string (nullable = true)
 |-- product: string (nullable = true)
 |-- amount: string (nullable = true)
 |-- order_date: string (nullable = true)
 |-- status: string (nullable = true)
 |-- city_clean: string (nullable = true)
 |-- category_clean: string (nullable = true)
 |-- product_clean: string (nullable = true)
 |-- amount_clean: string (nullable = true)
 |-- amount_int: integer (nullable = true)
 |-- order_date_clean: date (nullable = true)
 |-- order_value_category: string (nullable = true)
 |-- city: string (nullable = true)

root
 |-- city_clean: string (nullable = true)
 |-- total_revenue: long (nullable = true)



7

4. Compare size and performance against CSV

In [43]:
import os
import time
import builtins

csv_path = "/content/drive/MyDrive/Colab Notebooks/orders.csv"   #drive path
parquet_path = "orders_parquet"
orc_path = "city_revenue_orc"

print("\n--- File Size Comparison ---")

csv_file_size = os.path.getsize(csv_path)

parquet_dir_size = builtins.sum(
    f.stat().st_size for f in os.scandir(parquet_path) if f.is_file()
)

orc_dir_size = builtins.sum(
    f.stat().st_size for f in os.scandir(orc_path) if f.is_file()
)

print(f"CSV (orders.csv) size: {csv_file_size / (1024 * 1024):.2f} MB")
print(f"Parquet (orders_parquet) directory size: {parquet_dir_size / (1024 * 1024):.2f} MB")
print(f"ORC (city_revenue_orc) directory size: {orc_dir_size / (1024 * 1024):.2f} MB")

# --- Read Performance Comparison ---
print("\n--- Read Performance Comparison ---")

# CSV Read Time (Drive Path)
start_time = time.time()
spark.read \
    .option("header", "true") \
    .option("inferSchema", "false") \
    .csv(csv_path) \
    .count()
end_time = time.time()
print(f"CSV read time: {end_time - start_time:.4f} seconds")

# Parquet Read Time
start_time = time.time()
spark.read.parquet(parquet_path).count()
end_time = time.time()
print(f"Parquet read time: {end_time - start_time:.4f} seconds")

# ORC Read Time
start_time = time.time()
spark.read.orc(orc_path).count()
end_time = time.time()
print(f"ORC read time: {end_time - start_time:.4f} seconds")



--- File Size Comparison ---
CSV (orders.csv) size: 19.98 MB
Parquet (orders_parquet) directory size: 0.00 MB
ORC (city_revenue_orc) directory size: 0.00 MB

--- Read Performance Comparison ---
CSV read time: 0.7465 seconds
Parquet read time: 0.4430 seconds
ORC read time: 0.1706 seconds


##PHASE 12 – Debugging

Explain why this breaks:

df = df.filter(df.amount > 50000).show()

And why after this line df is no longer a DataFrame.

In [44]:
df = df.filter(df.amount > 50000).show()
#.show() returns None
#Assigning df with .show() overwrites it with None therefore pipline breaks

NameError: name 'df' is not defined

##PHASE 13 – Final Validation

1. Confrm:
- amount is IntegerType
- order_date_clean is DateType
- No nulls in critical business elds

In [45]:
print("--- Final Data Validation ---")

# Confirm amount_int is IntegerType and order_date is DateType
print("\nSchema verification for critical columns:")
df_final.select("amount_int", "order_date_clean").printSchema()

# Check for nulls in critical business fields (amount_int, order_date)
print("\nChecking for nulls in critical business fields (amount_int, order_date):")

# For amount_int
null_amount_count = df_final.filter(col("amount_int").isNull()).count()
print(f"Number of records with null amount_int: {null_amount_count}")

# For order_date
null_order_date_count = df_final.filter(col("order_date_clean").isNull()).count()
print(f"Number of records with null order_date: {null_order_date_count}")

if null_amount_count == 0 and null_order_date_count == 0:
    print("\nValidation successful: No nulls found in amount_int or order_date.")
else:
    print("\nValidation Alert: Nulls found in critical business fields.")

--- Final Data Validation ---

Schema verification for critical columns:
root
 |-- amount_int: integer (nullable = true)
 |-- order_date_clean: date (nullable = true)


Checking for nulls in critical business fields (amount_int, order_date):
Number of records with null amount_int: 23905
Number of records with null order_date: 2465

Validation Alert: Nulls found in critical business fields.


2. Document:
- Cleaning strategy
- Performance strategy
- Debugging learning

In [46]:
#Strategies and Learnings
#1. Cleaning Strategy

#StringType Ingestion: Load all columns as StringType to avoid schema inference issues and ensure complete data capture.
#Text Standardization: Trim spaces and apply proper case to city, category, and product columns.
#Amount Cleaning: Remove commas, validate numeric strings, cast to IntegerType; invalid values → NULL.
#Date Handling: Use coalesce with try_to_timestamp for multiple formats; unparseable dates → NULL.
#Preserve Raw Data: Create _clean columns for traceability.
#Deduplication: Remove duplicates based on order_id.
#Filter Valid Status: Keep only status = 'Completed'.

#2. Performance Strategy

#Partitioning: Monitor and repartition by common keys (e.g., city) to reduce shuffle.
#Broadcast Join: Use broadcast() for small lookup tables to avoid shuffling large DataFrames.
#Caching: Cache reused DataFrames for faster queries; demonstrated ~1.2x speedup.
#Unpersist: Free memory after use to prevent OOM issues.

#3. Debugging Learnings

#when().otherwise(): Ensure correct syntax and nesting.
#Date Parsing: Use try_to_timestamp instead of strict to_date.
#lit() for Constants: Wrap string literals to avoid AnalysisException.
#SQL Execution: Use spark.sql("query"), not spark.sql.session.sql().
#show() Behavior: show() is an action, returns None—don’t reassign DataFrame.
#sum() Conflict: Differentiate Python’s sum() vs PySpark’s sum().
#Handle None in RDDs: Replace None with defaults before aggregation.